In [1]:
from unstructured.partition.pdf import partition_pdf

# pdf_path = 'docs/Medical insurance.pdf'
# pdf_path = 'docs/doc3.pdf'
# pdf_path ='docs/Office address.pdf'
pdf_path = 'docs/table pdf.pdf'
# pdf_path = 'docs/pdf with image.pdf'
# Perseves Table Structure
elements = partition_pdf(pdf_path,strategy='hi_res',infer_table_structure=True) #only strategy tht does pictures and tables

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
for e in elements:
    print(type(e),e.text,e.metadata.category_depth)

<class 'unstructured.documents.elements.Title'> Example table This is an example of a data table. None
<class 'unstructured.documents.elements.Table'> Disability Category Participants Ballots Completed Blind 5 1 Low Vision 5 2 Dexterity 5 4 Mobility 3 3 Ballots Incomplete/ Terminated 4 3 1 0 Accuracy 34.5%, n=1 98.3% n=2 (97.7%, n=3) 98.3%, n=4 95.4%, n=3 Results Time to complete 1199 sec, n=1 1716 sec, n=3 (1934 sec, n=2) 1672.1 sec, n=4 1416 sec, n=3 None


In [383]:
elements[1].metadata.text_as_html

'<table><thead><th rowspan="2">sae pills) Category</th><th rowspan="2">Participants P</th><th rowspan="2">Eins Completed</th><th rowspan="2">Ballot: heer lete/ Ne Terminated</th><th colspan="2">Results</th></thead><thead><th></th><th></th><th></th><th></th><th>Accuracy</th><th>Time to complete</th></thead><tr><td>Blind</td><td>5</td><td>1</td><td>4</td><td>34.5%, n=1</td><td>1199 sec, n=1</td></tr><tr><td>Low Vision</td><td>5</td><td>2</td><td>3</td><td>98.3% n=2 (97.7%, n=3)</td><td>1716 sec, n=3 | (1934 sec, n=2)</td></tr><tr><td>Dexterity</td><td>5</td><td>4</td><td>1</td><td>98.3%, n=4</td><td>1672.1 sec, n=4</td></tr><tr><td>Mobility</td><td>3</td><td>3</td><td>0</td><td>95.4%, n=3</td><td>1416 sec, n=3</td></tr></table>'

In [3]:
from unstructured.cleaners.core import clean_bullets, clean_non_ascii_chars, clean_extra_whitespace

def preprocess_elements(elements):
    for e in elements:
        e.apply(clean_bullets)
        e.apply(clean_non_ascii_chars)
        e.apply(clean_extra_whitespace)
        if len(e.text) == 0:
            elements.remove(e)

In [4]:
preprocess_elements(elements)
for e in elements:
    print(type(e),e)

<class 'unstructured.documents.elements.Title'> Example table This is an example of a data table.
<class 'unstructured.documents.elements.Table'> Disability Category Participants Ballots Completed Blind 5 1 Low Vision 5 2 Dexterity 5 4 Mobility 3 3 Ballots Incomplete/ Terminated 4 3 1 0 Accuracy 34.5%, n=1 98.3% n=2 (97.7%, n=3) 98.3%, n=4 95.4%, n=3 Results Time to complete 1199 sec, n=1 1716 sec, n=3 (1934 sec, n=2) 1672.1 sec, n=4 1416 sec, n=3


In [5]:
from unstructured.documents.elements import Title, NarrativeText
from unstructured.staging.base import convert_to_dataframe

df = convert_to_dataframe(elements)



In [6]:
df[['text','type']]

,text,type
0,Example table This is an example of a data table.,Title
1,Disability Category Participants Ballots Compl...,Table


In [7]:
from unstructured.chunking.basic import chunk_elements
from unstructured.chunking.title import chunk_by_title


# def find_desired_chunking_length(elements):
#     max_length = 10000
#     paragraph_lengths = []
#     for e in elements:
#         metadata = e.to_dict()
#         if metadata['type'] =='NarrativeText':
#             paragraph_lengths.append(len(metadata['text']))
#     return min(paragraph_lengths)
            
# print(find_desired_chunking_length(elements))

# If we do chunk by title and don't make new_after_n_chars=0 it will try to fill up the max_characters window which will split addresses.pdf into 4 big chunks.
# if we do specify 0 for new_after_n_chars,  each element to appear in a chunk by itself, works well work doc3.pdf and office addresses.pdf but doesnt work well for lists.
# Chunking by title

# chunk_elements_pdf = chunk_elements(
#     elements=elements,
#     max_characters=1000,
#     new_after_n_chars=0
# )

# def page_chunking(elements):
#     final_chunks = []
#     paragraph_elements = []
#     other_elements = []

#     page_counter = 1
#     page_start = 0
#     use_chunk_by_title = False
#     for page_end in range(len(elements)):
#         print(page_end)
#         page_number = elements[page_end].to_dict()['metadata']['page_number']
#         if page_number > page_counter: 
#             print('new page',page_number,' go through prev page elements')
#             for e in range(page_start,page_end):
#                if elements[e].to_dict()['type'] != "NarrativeText":
#                    use_chunk_by_title = True
#                    break
#             if use_chunk_by_title:
#                 other_elements.extend(elements[page_start,page_end])
#             else:
#                 paragraph_elements.extend(elements[page_start,page_end])
            
#             page_counter+=1
#             page_start=page_end+1
#             use_chunk_by_title=False
'''
Alternative strategy since a lot of list items are misclassified as NarrativeText is to chunk by title if we find any lists in the pdf
'''
def smart_chunking(elements):
    paragraph_elements = []
    other_elements = []

    for e in elements:
        if e.to_dict()['type'] in ['NarrativeText']:
            paragraph_elements.append(e)
        else:
            other_elements.append(e)
    print('number of paragraphs',len(paragraph_elements))
    print('number of non-paragraph elements',len(other_elements))
    paragraph_chunks = chunk_elements(elements=paragraph_elements,max_characters=1000,new_after_n_chars=0)
    other_chunks = chunk_by_title(elements=other_elements,max_characters=1000)
    final_chunks = paragraph_chunks + other_chunks
    return final_chunks

def cheat_chunking(elements):
    chunk_with_title = False
    for e in elements:
        if e.to_dict()['type'] not in ['Title','NarrativeText','UncategorizedText']:
            chunk_with_title = True
    if chunk_with_title == True:
        return chunk_by_title(elements=elements,max_characters=1000)
    else:
        return chunk_elements(elements=elements,max_characters=1000,new_after_n_chars=0)
    
def openai_summarize_string(text):
    prompt = f"Summarize the given string (this can be a piece of text or html of a table) concisely in less than 50 characters: " + text
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Give what the user needs right away."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

            
chunk_elements_pdf = cheat_chunking(elements)
print(len(chunk_elements_pdf))
print(chunk_elements_pdf)

2
[<unstructured.documents.elements.CompositeElement object at 0x2ef700ca0>, <unstructured.documents.elements.Table object at 0x2ef700b80>]


In [8]:
for chunk in chunk_elements_pdf:
    print('--'*20)    
    print(chunk.metadata.orig_elements)
    print(chunk)
    print('--'*20)    

----------------------------------------
Example table This is an example of a data table.
----------------------------------------
----------------------------------------
Disability Category Participants Ballots Completed Blind 5 1 Low Vision 5 2 Dexterity 5 4 Mobility 3 3 Ballots Incomplete/ Terminated 4 3 1 0 Accuracy 34.5%, n=1 98.3% n=2 (97.7%, n=3) 98.3%, n=4 95.4%, n=3 Results Time to complete 1199 sec, n=1 1716 sec, n=3 (1934 sec, n=2) 1672.1 sec, n=4 1416 sec, n=3
----------------------------------------


In [9]:
chunk_elements_pdf[0].text = 'e'+ chunk_elements_pdf[0].text

In [10]:
def get_global_context_with_titles(elements):
    # Adding Global Context
    file_title =''
    element_metadata= elements[0].to_dict()
    if element_metadata['type'] == 'Title':
        file_title = elements[0].text
    file_name = element_metadata['metadata']['filename']
    header_injection = "Under \'"+file_title+"\' in file name \'"+file_name+'\''
    return header_injection

def add_titles_string(elements):
    # Adding a special chunk that gives the 'table of contents' by adding all titles
    titles_text = ""
    for e in elements:
        if e.to_dict()['type'] in ['Title']:
            titles_text+=e.to_dict()['text']
    return titles_text

def prepend_chunk_text(chunks,prepend_text):
    for c in chunks:
        c.text = prepend_text+' \n'+c.text


In [14]:
def custom_chunking_methods(chunks,elements,use_ai_summary=False):
    # Make sure for Tables, the text that will be vectorized later on is the summary of the table
    # not the html too semantically more dense
    for c in chunks:
        c_data = c.to_dict()
        if c_data['type'] == "Table":
            c.text = openai_summarize_string(c_data['metadata']['text_as_html'])

    global_context_prepend = None
    if use_ai_summary:
        # use chatgpt summary of the full document
        full_doc_text = '\n'.join([e.text for e in elements])
        global_context_prepend = openai_summarize_string(full_doc_text)
    else: 
        #get global context by the file name and titles
        global_context_prepend = get_global_context_with_titles(elements)

    # get global context by the summary 
    prepend_chunk_text(chunks,global_context_prepend)

custom_chunking_methods(chunk_elements_pdf,elements,use_ai_summary=True)

In [15]:
for c in chunk_elements_pdf:
    print(c.text)
    print()

Disability categories and ballot results table. 
eExample table This is an example of a data table.

Disability categories and ballot results table. 
Summary of Table: Participant data on accuracy and completion time.



In [16]:
import os
from dotenv import load_dotenv
from openai import OpenAI
client = OpenAI()
from unstructured.documents.elements import Text
from unstructured.embed.openai import OpenAIEmbeddingConfig, OpenAIEmbeddingEncoder

load_dotenv()
# Initialize the encoder with OpenAI credentials and using ada-2
embedding_encoder = OpenAIEmbeddingEncoder(config=OpenAIEmbeddingConfig(api_key=os.getenv('OPENAI_API_KEY')))


In [17]:
def create_embeddings(chunks):
    chunk_embeddings = embedding_encoder.embed_documents(
        elements=chunks,
    )

    # # Embed a single query string
    # query = "This is the query"
    # query_embedding = embedding_encoder.embed_query(query=query)

    # Print embeddings
    [print(e.embeddings[:5], e.to_dict()) for e in chunk_embeddings]
    # print(query_embedding, query)
    print('number of embeddings created by chunks',len(chunk_embeddings))
    print(embedding_encoder.is_unit_vector(), embedding_encoder.num_of_dimensions())
    return chunk_embeddings

embedding_chunks = create_embeddings(chunk_elements_pdf)

[-0.008442073948447447, 0.0006493248872789692, 0.0009114376772693561, -0.01623056814654431, -0.015781232999500742] {'type': 'CompositeElement', 'element_id': '40ba7b99-e861-4808-8a09-ec9059daa985', 'text': 'Disability categories and ballot results table. \neExample table This is an example of a data table.', 'metadata': {'file_directory': 'docs', 'filename': 'table pdf.pdf', 'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-05-30T14:12:39', 'page_number': 1, 'orig_elements': 'eJxlUUuPgyAQ/iuEc+vysLX2vvdNtjfTmBHGSoJAlGZrmv3vC9aeNoHD95hh5qN5UrQ4oout0fRM6OlYllIxVcu6O2APvBeyRMlRlnUlUNEdoSNG0BAh+Z9UeT9p4yDivGILi7/HdkBzG2JihGAs1Wz0j9FxSCyvVjZ442Kuaxpe1wXfEVHxorruyBvLY1mwjE+CF/I/fvkTQedljjjmHb7MA+13AIX0NwkaI6povGuVhXluw+S7ZGNFyU6nYzL0xmKrzZRcflpyB+3VTDfFwYiZi9BZJEH3RbpvMS5hFSEEaxTkVz422YK73eG2xtJQdDd6Xdk5tqPXpje4Bi6YKPfssJfswsszF2dZ5+qQKlt3Hzucclx5j4iPHCj9fMAY0iivgS6DmUk64Ahugu8JkPxBL0uR+70HvZhoUyzXP9cKmBQ='}, 'embeddings': [-0.008442073948447447, 0.00064932488727896

In [18]:
embedding_chunks

In [19]:
import faiss
import numpy as np

def store_chunks_in_faiss(chunks):
    raw_embeddings = []
    for element in chunks:
        raw_embeddings.append(element.embeddings)

    embeddings_np = np.array(raw_embeddings).astype('float32')
    # Create FAISS index
    dimension = embeddings_np.shape[1]
    faiss_index = faiss.IndexFlatL2(dimension)
    faiss_index.add(embeddings_np)

    return faiss_index


faiss_index = store_chunks_in_faiss(chunk_elements_pdf)

In [23]:
# Function to query FAISS and retrieve corresponding CompositeElements
def query_faiss(query, faiss_index, composite_elements):
    # Generate embedding for the query
    query_embedding = embedding_encoder.embed_query(query=query)
    query_embedding_np = np.array([query_embedding]).astype('float32')

    # Search FAISS index
    distances, indices = faiss_index.search(query_embedding_np, k=5)  # Top 5 matches

    # Retrieve the corresponding CompositeElements
    results = [(composite_elements[idx], distances[0][i]) for i, idx in enumerate(indices[0])]

    return results


def range_query_faiss(query, faiss_index, composite_elements, threshold):
    # Generate embedding for the query
    query_embedding = embedding_encoder.embed_query(query=query)
    query_embedding_np = np.array([query_embedding]).astype('float32')

    # Initialize parameters for range search
    radius = threshold
    lims, D, I = faiss_index.range_search(query_embedding_np, radius)

    # Retrieve the corresponding CompositeElements and their distances
    results = []
    for i in range(len(I)):
        if D[i] < threshold:  # Apply additional check to ensure threshold is respected
            results.append((composite_elements[I[i]], D[i]))

    return results



query = "What's the accuracy of vision"
results = query_faiss(query, faiss_index, chunk_elements_pdf)


threshold = 1.0  # Distance threshold
range_results = range_query_faiss(query, faiss_index, chunk_elements_pdf, threshold)

for result, distance in results:
    print("Metadata:", result.metadata)
    print("Score (Distance):", distance)
    print("text:", result)

print(len(range_results))
for result,distance in range_results:
    print("Score (Distance):", distance)
    print("Metadata:", result.metadata.to_dict())
    print("text:", result)


Metadata: <unstructured.documents.elements.ElementMetadata object at 0x2ef700d90>
Score (Distance): 0.47347417
text: Disability categories and ballot results table. 
Summary of Table: Participant data on accuracy and completion time.
Metadata: <unstructured.documents.elements.ElementMetadata object at 0x2ef700cd0>
Score (Distance): 0.56468487
text: Disability categories and ballot results table. 
eExample table This is an example of a data table.
Metadata: <unstructured.documents.elements.ElementMetadata object at 0x2ef700d90>
Score (Distance): 3.4028235e+38
text: Disability categories and ballot results table. 
Summary of Table: Participant data on accuracy and completion time.
Metadata: <unstructured.documents.elements.ElementMetadata object at 0x2ef700d90>
Score (Distance): 3.4028235e+38
text: Disability categories and ballot results table. 
Summary of Table: Participant data on accuracy and completion time.
Metadata: <unstructured.documents.elements.ElementMetadata object at 0x2ef7

In [32]:
def generate_answer(relevant_chunks_text, query):
    prompt = f"Based on the following chunks from the document, answer the query: {query}\n\n" + "\n\n".join(relevant_chunks_text)
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are given documents to help answer queries."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content


def rag_answer(query):
    range_results = range_query_faiss(query, faiss_index, chunk_elements_pdf, threshold)
    print(range_results)
    # chunk_ids = [result.id for result, _ in range_results]
    relevant_chunks_text = [chunk.text for chunk,distance in range_results]
    for chunk, distance in range_results:
        if chunk.to_dict()['type'] == 'Table':
            relevant_chunks_text.append(chunk.to_dict()['metadata']['text_as_html'])
        else:
            relevant_chunks_text.append(chunk.text)
    print(relevant_chunks_text)
    return generate_answer(relevant_chunks_text, query)

rag_answer('What is the time to complete for mobility category of the study')

[(<unstructured.documents.elements.CompositeElement object at 0x2ef700ca0>, 0.4427842), (<unstructured.documents.elements.Table object at 0x2ef700b80>, 0.34332806)]
['Disability categories and ballot results table. \neExample table This is an example of a data table.', 'Disability categories and ballot results table. \nSummary of Table: Participant data on accuracy and completion time.', 'Disability categories and ballot results table. \neExample table This is an example of a data table.', '<table><thead><th rowspan="2">sae pills) Category</th><th rowspan="2">Participants P</th><th rowspan="2">Eins Completed</th><th rowspan="2">Ballot: heer lete/ Ne Terminated</th><th colspan="2">Results</th></thead><thead><th></th><th></th><th></th><th></th><th>Accuracy</th><th>Time to complete</th></thead><tr><td>Blind</td><td>5</td><td>1</td><td>4</td><td>34.5%, n=1</td><td>1199 sec, n=1</td></tr><tr><td>Low Vision</td><td>5</td><td>2</td><td>3</td><td>98.3% n=2 (97.7%, n=3)</td><td>1716 sec, n=3 | 

'The time to complete for the mobility category of the study is 1416 seconds, with \\( n = 3 \\).'

In [362]:
llm_answer = rag_answer("What's the accuray of vision")

In [363]:
llm_answer.content

'Based on the document provided, Microsoft has offices in the following cities:\n\n1. Boston\n2. Dallas\n3. Los Angeles\n4. Houston\n5. Miami\n6. Seattle\n7. Denver\n8. Phoenix\n9. San Diego\n10. Detroit\n11. Philadelphia\n12. Minneapolis\n13. Charlotte\n14. Orlando\n15. Portland\n16. Pittsburgh\n17. Cincinnati\n18. St. Louis\n19. Tampa\n20. Baltimore\n21. Indianapolis\n22. Kansas City\n23. Nashville\n24. Austin'

In [375]:
load_dotenv()

True

In [504]:
import mysql.connector
def connect_to_database():
    try:
        # Connect to MySQL database
        connection = mysql.connector.connect(
            host='localhost',
            database='pdf_chunks',
            user='root',
            password=os.getenv('MYSQL_PASS')
        )
        return connection
    except Error as e:
        print(f"Error: {e}")

def create_database_and_table(connection):
    try:
        if connection.is_connected():
            cursor = connection.cursor()

            # Create database
            cursor.execute("CREATE DATABASE IF NOT EXISTS pdf_chunks;")
            print("Database `pdf_chunks` created or already exists.")

            # Select the newly created database
            cursor.execute("USE pdf_chunks;")
            # Create table if not exists
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS chunks_1 (
                element_id VARCHAR(255) PRIMARY KEY,
                content TEXT
            );
            """)

            print("Table `chunks_1` created or already exists.")

    except Error as e:
        print(f"Error: {e}")

def query_chunk_by_ids(connection, element_ids):
    try:
        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)

            # Create a string of placeholders for element_ids
            placeholders = ', '.join(['%s'] * len(element_ids))

            # Query to retrieve the chunks by element_ids
            sql_select_query = f"SELECT * FROM chunks_1 WHERE element_id IN ({placeholders})"
            cursor.execute(sql_select_query, element_ids)

            # Fetch all the results
            results = cursor.fetchall()

            if results:
                print("Chunks found:")
                for result in results:
                    print(f"Element ID: {result['element_id']}")
                    print(f"Content: {result['content']}")
                return results
            else:
                print("No chunks found with the given element IDs.")

    except Error as e:
        print(f"Error: {e}")

def insert_chunk(connection, element_id, content):
    try:
        if connection.is_connected():
            cursor = connection.cursor()

            # Insert chunk into the table
            sql_insert_query = """
            INSERT INTO chunks_1 (element_id, content)
            VALUES (%s, %s);
            """
            cursor.execute(sql_insert_query, (element_id, content))
            connection.commit()
            print("Chunk inserted successfully.")
        else:
            print('Connection is not connected')
    except Error as e:
        print(f"Error: {e}")

def view_table_contents(connection):
    try:
        if connection.is_connected():
            cursor = connection.cursor(dictionary=True)

            # Select all rows from the table
            sql_select_query = "SELECT * FROM chunks_1"
            cursor.execute(sql_select_query)

            # Fetch all rows
            rows = cursor.fetchall()

            if rows:
                return rows
            else:
                print("The table is empty.")
        else:
            print('Not connected')

    except Error as e:
        print(f"Error: {e}")



In [487]:
connection = connect_to_database()
create_database_and_table(connection)

Database `pdf_chunks` created or already exists.
Table `chunks_1` created or already exists.


In [481]:
for chunk in chunk_elements_pdf:
    if chunk.to_dict()['type'] == 'Table':
        print('Table detected - storing raw html instead of text')
        insert_chunk(connection,chunk.id,chunk.to_dict()['metadata']['text_as_html'])
    else:
        insert_chunk(connection, chunk.id, chunk.text)


Chunk inserted successfully.
Table detected - storing raw html instead of text
Chunk inserted successfully.


In [488]:
# Call the function to view table contents
view_table_contents(connection)

[{'element_id': '5d26b638-50b2-4e1d-9400-25f32f6c80f6',
  'content': '<table><thead><th rowspan="2">sae pills) Category</th><th rowspan="2">Participants P</th><th rowspan="2">Eins Completed</th><th rowspan="2">Ballot: heer lete/ Ne Terminated</th><th colspan="2">Results</th></thead><thead><th></th><th></th><th></th><th></th><th>Accuracy</th><th>Time to complete</th></thead><tr><td>Blind</td><td>5</td><td>1</td><td>4</td><td>34.5%, n=1</td><td>1199 sec, n=1</td></tr><tr><td>Low Vision</td><td>5</td><td>2</td><td>3</td><td>98.3% n=2 (97.7%, n=3)</td><td>1716 sec, n=3 | (1934 sec, n=2)</td></tr><tr><td>Dexterity</td><td>5</td><td>4</td><td>1</td><td>98.3%, n=4</td><td>1672.1 sec, n=4</td></tr><tr><td>Mobility</td><td>3</td><td>3</td><td>0</td><td>95.4%, n=3</td><td>1416 sec, n=3</td></tr></table>'},
 {'element_id': 'b756d11d-8b81-48cc-b9b3-5eb017e54361',
  'content': 'Data table: Participant categories, completion, and results. \nStudy Results: Blind, Low Vision, Dexterity, Mobility DataE

In [511]:
def generate_answer(relevant_chunks_text, query):
    prompt = f"Based on the following chunks from the document, answer the query: {query}\n\n" + "\n\n".join(relevant_chunks_text)
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are given documents to help answer queries."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content


def rag_answer(query):
    range_results = range_query_faiss(query, faiss_index, chunk_elements_pdf, threshold)
    chunk_ids = [result.id for result, _ in range_results]
    print(chunk_ids)
    connection = connect_to_database()
    chunks = query_chunk_by_ids(connection,chunk_ids)
    relevant_chunks_text = [c['content'] for c in chunks]
    return generate_answer(relevant_chunks_text, query)

rag_answer('What is the time to completeness in Mobility study?')

['b756d11d-8b81-48cc-b9b3-5eb017e54361', '5d26b638-50b2-4e1d-9400-25f32f6c80f6']
Chunks found:
Element ID: 5d26b638-50b2-4e1d-9400-25f32f6c80f6
Content: <table><thead><th rowspan="2">sae pills) Category</th><th rowspan="2">Participants P</th><th rowspan="2">Eins Completed</th><th rowspan="2">Ballot: heer lete/ Ne Terminated</th><th colspan="2">Results</th></thead><thead><th></th><th></th><th></th><th></th><th>Accuracy</th><th>Time to complete</th></thead><tr><td>Blind</td><td>5</td><td>1</td><td>4</td><td>34.5%, n=1</td><td>1199 sec, n=1</td></tr><tr><td>Low Vision</td><td>5</td><td>2</td><td>3</td><td>98.3% n=2 (97.7%, n=3)</td><td>1716 sec, n=3 | (1934 sec, n=2)</td></tr><tr><td>Dexterity</td><td>5</td><td>4</td><td>1</td><td>98.3%, n=4</td><td>1672.1 sec, n=4</td></tr><tr><td>Mobility</td><td>3</td><td>3</td><td>0</td><td>95.4%, n=3</td><td>1416 sec, n=3</td></tr></table>
Element ID: b756d11d-8b81-48cc-b9b3-5eb017e54361
Content: Data table: Participant categories, completion, and re

'The time to completeness in the Mobility study is 1416 seconds for the 3 participants who completed it.'